## Import all the important libraries

In [3]:
import os
import numpy as np
import soundfile as sf
import librosa as lbs
import scipy 
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, classification_report

## Normalized the audio signals 

In [ ]:
# Normalizing the bus signals
#input_folder = "Bus Testing Set"
#output_folder = "Normalized Testing Bus Sounds"
#os.makedirs(output_folder, exist_ok=True)
#for file in os.listdir(input_folder):
#    if file.endswith(".wav"):
#        path = os.path.join(input_folder, file)
#        x, sr = sf.read(path)
#        x = x.astype(np.float32)
#        # Standardization
#        mean = np.mean(x)
#        std = np.std(x)
#        if std < 1e-8:
#            x_norm = x - mean
#        else:
#            x_norm = (x - mean) / std
#        # Save normalized file
#        out_path = os.path.join(output_folder, file)
#        sf.write(out_path, x_norm, sr)
#
## Normalizing the tram sounds
#input_folder = "Tram Testing Set"
#output_folder = "Normalized Testing Tram Sounds"
#os.makedirs(output_folder, exist_ok=True)
#for file in os.listdir(input_folder):
#    if file.endswith(".wav"):
#        path = os.path.join(input_folder, file)
#        x, sr = sf.read(path)
#        x = x.astype(np.float32)
#        # Standardization
#        mean = np.mean(x)
#        std = np.std(x)
#        if std < 1e-8:
#            x_norm = x - mean
#        else:
#            x_norm = (x - mean) / std
#        # Save normalized file
#        out_path = os.path.join(output_folder, file)
#        sf.write(out_path, x_norm, sr)

## Feature extraction from normalized audio

Run this after normalization. It reads `Normalized Bus Sounds` `Normalized Tram Sounds
`, computes energy, RMS, power spectrogram, mel spectrogram, MFCC, and CQT

In [ ]:
# Extract features from normalized audio

#input_root = "Normalized Dataset Testing"
#classes = {"bus": 0, "tram": 1}
#
#
#feature_root = "Extracted_Features_Testing"
#os.makedirs(feature_root, exist_ok=True)
#
#
#n_fft = 2048
#hop = 512
#n_mels = 40
#n_mfcc = 24
#
#for label_name, label_id in classes.items():
#    input_folder = os.path.join(input_root, label_name)
#    output_folder = os.path.join(feature_root, label_name)
#    os.makedirs(output_folder, exist_ok=True)
#    
#    for file in sorted(os.listdir(input_folder)):
#        if not file.endswith(".wav"):
#            continue
#
#        path = os.path.join(input_folder, file)
#
#        y, sr = lbs.load(path, sr=None, mono=True)
#
#        # Feature extraction
#        energy = np.sum(y ** 2)
#
#        rms = lbs.feature.rms(y=y, frame_length=n_fft, hop_length=hop).squeeze()
#
#        power_spec = np.abs(lbs.stft(y, n_fft=n_fft, hop_length=hop)) ** 2
#
#        mel_spec = lbs.feature.melspectrogram(
#            S=power_spec, sr=sr, n_mels=n_mels
#        )
#
#        mfcc = lbs.feature.mfcc(
#            S=lbs.power_to_db(mel_spec), sr=sr, n_mfcc=n_mfcc
#        )
#
#        cqt = np.abs(lbs.cqt(
#            y=y, sr=sr, hop_length=hop, n_bins=84, bins_per_octave=12
#        ))
#
#        # Save into file for ML model
#        base = file.replace(".wav", "")
#
#        np.save(os.path.join(output_folder, f"{base}_energy.npy"), energy)
#        np.save(os.path.join(output_folder, f"{base}_rms.npy"), rms)
#        np.save(os.path.join(output_folder, f"{base}_mel.npy"), mel_spec)
#        np.save(os.path.join(output_folder, f"{base}_mfcc.npy"), mfcc)
#        np.save(os.path.join(output_folder, f"{base}_cqt.npy"), cqt)


## Defining ML Model for classifying the Audio

In [4]:
# Function to build feature vector
def build_feature_vector(folder, base_name):
    energy = np.load(os.path.join(folder, f"{base_name}_energy.npy"))
    rms    = np.load(os.path.join(folder, f"{base_name}_rms.npy"))
    mel    = np.load(os.path.join(folder, f"{base_name}_mel.npy"))
    mfcc   = np.load(os.path.join(folder, f"{base_name}_mfcc.npy"))
    cqt    = np.load(os.path.join(folder, f"{base_name}_cqt.npy"))

    feats = []

    # scalar
    feats.append(float(energy))

    # rms mean/std
    feats.append(np.mean(rms))
    feats.append(np.std(rms))

    # mfcc mean/std
    feats.extend(np.mean(mfcc, axis=1))
    feats.extend(np.std(mfcc, axis=1))

    # mel mean/std
    feats.extend(np.mean(mel, axis=1))
    feats.extend(np.std(mel, axis=1))

    # cqt mean/std
    feats.extend(np.mean(cqt, axis=1))
    feats.extend(np.std(cqt, axis=1))

    return np.array(feats, dtype=np.float32)

# Function to load split in order to define traning and validating samples

def load_split(split_folder):
    X = []
    y = []

    classes = {"bus": 0, "tram": 1}

    for class_name, label in classes.items():
        class_folder = os.path.join(split_folder, class_name)

        for file in sorted(os.listdir(class_folder)):
            if file.endswith("_energy.npy"):
                base = file.replace("_energy.npy", "")
                vec = build_feature_vector(class_folder, base)
                X.append(vec)
                y.append(label)

    return np.array(X), np.array(y)

# Loading the training, validating and testing sets

feature_root = "Extracted_Features"

X_train, y_train = load_split(os.path.join(feature_root, "training"))
X_val,   y_val   = load_split(os.path.join(feature_root, "validating"))
X_test,  y_test  = load_split(os.path.join(feature_root, "testing"))

print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)


(160, 299) (160,)
(20, 299) (20,)
(20, 299) (20,)


## Running hyperparameters tuning to determine best parameters

In [5]:
C_list = [0.1, 1, 10, 100]
gamma_list = ["scale", 0.1, 0.01, 0.001]

best_val_acc = 0
best_params = None

for C in C_list:
    for gamma in gamma_list:

        model = make_pipeline(
            StandardScaler(),
            SVC(kernel='rbf', C=C, gamma=gamma)
        )

        model.fit(X_train, y_train)

        val_pred = model.predict(X_val)
        val_acc = accuracy_score(y_val, val_pred)

        print(f"C={C}, gamma={gamma} -> val_acc={val_acc:.4f}")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_params = (C, gamma)

print("\nBest parameters:", best_params)
print("Best validation accuracy:", best_val_acc)

C=0.1, gamma=scale -> val_acc=0.9500
C=0.1, gamma=0.1 -> val_acc=0.5000
C=0.1, gamma=0.01 -> val_acc=0.7000
C=0.1, gamma=0.001 -> val_acc=0.9500
C=1, gamma=scale -> val_acc=0.9500
C=1, gamma=0.1 -> val_acc=0.5000
C=1, gamma=0.01 -> val_acc=0.9500
C=1, gamma=0.001 -> val_acc=0.9500
C=10, gamma=scale -> val_acc=0.9500
C=10, gamma=0.1 -> val_acc=0.5000
C=10, gamma=0.01 -> val_acc=0.9500
C=10, gamma=0.001 -> val_acc=0.9500
C=100, gamma=scale -> val_acc=0.9500
C=100, gamma=0.1 -> val_acc=0.5000
C=100, gamma=0.01 -> val_acc=0.9500
C=100, gamma=0.001 -> val_acc=0.9500

Best parameters: (0.1, 'scale')
Best validation accuracy: 0.95


## Training model based on the best parameters determined

In [11]:
best_C, best_gamma = best_params

final_model = make_pipeline(
    StandardScaler(),
    SVC(kernel='rbf', C=best_C, gamma=best_gamma)
)

final_model.fit(X_train, y_train)

,steps,"[('standardscaler', ...), ('svc', ...)]"
,transform_input,None
,memory,None
,verbose,False
,copy,True
,with_mean,True
,with_std,True
,C,0.1
,kernel,'rbf'
,degree,3
,gamma,'scale'


## Evaluate on the test set

In [12]:
y_test_pred = final_model.predict(X_test)

print("TEST Accuracy:", accuracy_score(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred))

TEST Accuracy: 0.85
              precision    recall  f1-score   support

           0       1.00      0.70      0.82        10
           1       0.77      1.00      0.87        10

    accuracy                           0.85        20
   macro avg       0.88      0.85      0.85        20
weighted avg       0.88      0.85      0.85        20

